<a href="https://colab.research.google.com/github/MaimunaSun/NLP/blob/main/Notebook/Model_Develoment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Restaurants receive thousands of customer reviews, but star ratings alone fail to explain why ratings increase or decline. Reviews often contain rich information about food quality, service, pricing, ambience, and operational factors, yet this information remains unstructured and difficult to analyze at scale. This limits restaurants’ ability to identify the drivers of customer satisfaction and dissatisfaction

Notebook 1: Model Development

This notebook develops models for sentiment prediction and aspect extraction

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
print("PySpark installed and imported successfully!")

PySpark installed and imported successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YelpAnalysis") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()


Loading the data

In [ ]:
business_path = "/content/drive/MyDrive/yelp_dataset/yelp_academic_dataset_business.json"

business_df = spark.read.json(business_path)
business_df.printSchema()


root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [ ]:
review_path = "/content/drive/MyDrive/yelp_dataset/yelp_academic_dataset_review.json"

reviews_df = spark.read.json(review_path)
reviews_df.printSchema()


root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



Cleand and Filter Data

In [ ]:
#mandatory
from pyspark.sql.functions import col

business_df_clean = business_df.filter(
    col("business_id").isNotNull() &
    col("categories").isNotNull()
)


In [ ]:
#Only need restaurant data
restaurants_df = business_df_clean.filter(
    col("categories").contains("Restaurants")
)

In [ ]:
#mandatory
reviews_df_clean = reviews_df.filter(
    col("review_id").isNotNull() &
    col("business_id").isNotNull() &
    col("text").isNotNull() &
    col("stars").isNotNull()
)


In [ ]:
#Sample Review
reviews_sample = reviews_df_clean.sample(fraction=0.05, seed=42)

In [ ]:
#Only restaurent reviews are required
restaurant_reviews_df = reviews_sample.join(
    restaurants_df.select("business_id"),
    on="business_id",
    how="inner"
)



Select Useful Fields for Sentiment Predeiction

In [ ]:
sentiment_df = restaurant_reviews_df.select("review_id", "business_id", "text", "stars")

In [ ]:
from pyspark.sql.functions import col, when

sentiment_df = sentiment_df.withColumn(
    "label",
    when(col("stars") <= 2, 0)  # Negative
    .when(col("stars") == 3, 1)  # Neutral
    .otherwise(2)  # Positive
)



In [ ]:
!pip install transformers datasets accelerate


Convert to Hugging Face Dataset

In [ ]:
from datasets import Dataset

# Keep only relevant columns
hf_dataset = Dataset.from_spark(sentiment_df.select("text", "label"))

Loading dataset shards:   0%|          | 0/40 [00:00<?, ?it/s]

Tokenize

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

hf_dataset = hf_dataset.map(tokenize, batched=True)
hf_dataset = hf_dataset.remove_columns(["text"])
hf_dataset.set_format("torch")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/235995 [00:00<?, ? examples/s]

Train / Evaluation Split

In [ ]:
hf_dataset = hf_dataset.shuffle(seed=42)

split_index = int(0.8 * len(hf_dataset))
train_dataset = hf_dataset.select(range(split_index))
eval_dataset = hf_dataset.select(range(split_index, len(hf_dataset)))


Initialize DistilBERT for classification

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=3  # Negative, Neutral, Positive
)



model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training Arguments

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="no",
    report_to="none"
)


Evaluate arguments

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

Initialize Trainer

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)


Train Model

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.320500,0.312321,0.881544,0.871254,0.868526,0.881544
2,0.251800,0.308882,0.883959,0.882144,0.880643,0.883959
3,0.179600,0.391490,0.882201,0.880399,0.878851,0.882201


TrainOutput(global_step=35400, training_loss=0.266321577944998, metrics={'train_runtime': 6584.1513, 'train_samples_per_second': 86.023, 'train_steps_per_second': 5.377, 'total_flos': 1.875732075171533e+16, 'train_loss': 0.266321577944998, 'epoch': 3.0})

Evaluate

In [ ]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.39149010181427, 'eval_accuracy': 0.8822008940867391, 'eval_f1': 0.8803987406994876, 'eval_precision': 0.8788513945633782, 'eval_recall': 0.8822008940867391, 'eval_runtime': 166.3441, 'eval_samples_per_second': 283.743, 'eval_steps_per_second': 17.734, 'epoch': 3.0}


save sentiment prediction model

In [ ]:
MODEL_DIR = "/content/drive/MyDrive/restaurant_sentiment_model"

trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)



('/content/drive/MyDrive/restaurant_sentiment_model/tokenizer_config.json',
 '/content/drive/MyDrive/restaurant_sentiment_model/special_tokens_map.json',
 '/content/drive/MyDrive/restaurant_sentiment_model/vocab.txt',
 '/content/drive/MyDrive/restaurant_sentiment_model/added_tokens.json',
 '/content/drive/MyDrive/restaurant_sentiment_model/tokenizer.json')

ASPECT EXTRACTION

Get reviews + business_id

In [ ]:
reviews_pdf = sentiment_df.select("business_id", "text").toPandas()[:10000]
reviews_texts = reviews_pdf["text"].tolist()

In [ ]:
pip install -U bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 6.5 MB/s eta 0:00:00


aspect extraction

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

topic_model = BERTopic(
    embedding_model=embedding_model,
    verbose=True
)

topics, probs = topic_model.fit_transform(reviews_texts)

#SAVE MODEL

MODEL_PATH = "/content/drive/MyDrive/restaurant_sentiment_model/aspect"
topic_model.save(
    MODEL_PATH,
    serialization="pickle"
)



2025-12-25 09:41:39,896 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

2025-12-25 09:41:54,868 - BERTopic - Embedding - Completed ✓
2025-12-25 09:41:54,869 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-25 09:42:12,297 - BERTopic - Dimensionality - Completed ✓
2025-12-25 09:42:12,298 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-25 09:42:12,728 - BERTopic - Cluster - Completed ✓
2025-12-25 09:42:12,734 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-25 09:42:13,419 - BERTopic - Representation - Completed ✓
2025-12-25 09:42:13,947 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
